# Getting Real with Real Estate

Hold for intro paragraph

### Load Data

In [2]:
#Libraries needed
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
og=pd.read_csv('train.csv', sep=',')
df_iowa = og.copy()

og_slo=pd.read_csv('RealEstate.csv', sep =',')
df_slo = og_slo.copy()

og_nc=pd.read_excel('woodard.xls')
df_nc = og_nc.copy()

## Data Exploration

## Handaling Missing Values and Outliers

Hold for explanation

## Engineering Features

## Identify Valuable Relationships

### Bed/Bath and Sales Price

## Confirm Trend